# Objectives

-	Increase dpi and redo conversion CHECK 
-	Require a publisher (will avoid hand-drawn) CHECK
-	Top 50 composers for now in toy dataset
-	Elim orchestral scores
-	Make sure there aren’t cover pages, etc.
-	Make sure data in grayscale—standardize
-	Do as much of this as possible in an automated fashion. Publisher requirement will require some iteration and testing. 
-	ALSO re-do cell phone pictures, taking into account these constraints.


In [1]:
import glob
import os
import subprocess
import numpy as np
from scipy import ndimage
from scipy import signal
from scipy.misc import imsave
from scipy.signal import convolve2d
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage.transform import rotate, resize
import time
import pylab as pl
from IPython import display
import librosa
import cPickle
import pandas as pd

ModuleNotFoundError: No module named 'librosa'

## Convert w 300 dpi

In [ ]:
root_dir = 'score_data'
pdf_dir = root_dir + '/prepped_pdf' # directory containing pdfs of scores
png_dir = root_dir + '/prepped_png' # root output directory for png files

In [ ]:
#os.mkdir(png_dir)
pdf_files = glob(pdf_dir + '/*.pdf')
for pdf_file in pdf_files:
    basename = os.path.splitext(os.path.basename(pdf_file))[0]
    pngout = png_dir + '/' + basename + '.png'
    subprocess.call(['convert', '-density', '300', pdf_file, pngout]) # 72 dpi is the default value

## Rotate

In [ ]:
def importImage(pngfile):
    img = ndimage.imread(pngfile, flatten=True)
    img = 1 - img/255.0 # make white -> 0, black -> 1
    return img

In [ ]:
def showImage(X, sz = (12,12)):
    plt.figure(figsize=sz)
    plt.imshow(1-X, cmap='gray', vmin=0, vmax=1)

In [ ]:
def rotateToHorizontal(img, lb=-2, ub=2, incr=.01, topN=40):
    bestscore = -np.inf
    bestTheta = 0
    for theta in np.arange(lb, ub, incr):
        imgRot = rotate(img,theta)
        rsums = np.sum(imgRot, axis=1)
        rsums_sorted = sorted(rsums)[::-1]
        curscore = np.sum(rsums_sorted[0:topN])
        if curscore > bestscore:
            bestscore = curscore
            bestTheta = theta
    result = rotate(img,bestTheta)
    return result, bestTheta

In [ ]:
I = importImage('score_data/prepped_png/score17.png')
Irot, theta = rotateToHorizontal(I,-1,1)
showImage(Irot)

In [ ]:
def rotateAllImages(indir, outdir):
    png_files = glob.glob(indir + '/*.png')
    for png_file in png_files:
        I = importImage(png_file)
        Irot, theta = rotateToHorizontal(I)
        img = 255 - np.uint8(Irot*255)
        basename = os.path.basename(png_file)
        pngout = outdir + '/' + basename
        imsave(pngout,img)

In [ ]:
pngrot_dir = root_dir + '/toAnnotate' # rotated png files

In [ ]:
os.mkdir(pngrot_dir)
rotateAllImages(png_dir, pngrot_dir)

Here's an example of something w/o publisher: https://imslp.org/wiki/Acis_and_Galatea%2C_HWV_49_(Handel%2C_George_Frideric)
just go to the page parts and it's the first thing w/ id 106106

## Get bad (no publisher) names

We'll assume that we first loaded our text file as html text so we can parse with beautifulsoup. soup is the name of the object containing html info

In [ ]:
# open html text file as a beautifulsoup object
with open('html.txt') as f:
    soup = BeautifulSoup(f, "html.parser")

In [ ]:
for div in soup.find_all('div', class_ = 'we'):
    # we will contain both an ID and the publisher info for a piece so we can make sure it's not handwritten
    div = str(div)
    if 'Manuscript' in div or 'manuscript' in div:
        value_on = div[div.find('indexes='):] # this is where the ID should show up on a page
        bad_name = value_on[8:14] # just get the id, assuming 6 digits
        if bad_name[-1] = '/':
            # if it's a slash, there's something else after it
            bad_name_two = value_on[14:20]
            # check if last thing is actually a digit, in case of 5-digit ID
            if not bad_name_two[-1].isdigit():
                bad_name_two = bad_name_two[:-1]
            bad_pdf_two = bad_name_two + '.pdf'
            bad_names.append(bad_pdf_two)
        if not bad_name[-1].isdigit():
            # make could be a 5-digit ID
            bad_name = bad_name[:-1]
        bad_pdf = bad_name + '.pdf' # add to .pdf so we have the actual filename we want to avoid/delete if it's in our thing
        bad_names.append(bad_pdf) # append this to a list of bad names

## Check if Orchestral Score

Let's see if we can automate checking if smth is an orchestral score or just a part. Let's do a Beethoven Symphony :D

In [ ]:
divList = []
score_names = []

In [ ]:
for div in soup.find_all('div', class_ = 'we'):
    # we will contain both an ID and the publisher info for a piece so we can make sure it's not handwritten
    div = str(div)
    if 'Score' in div or 'score' in div:
        divList.append(div)

In [ ]:
for div in divList:
    value_on = div[div.find('indexes='):]
    score_name = value_on[8:14]
    if score_name[-1] = '/':
            # if it's a slash, there's something else after it
            score_name_two = value_on[14:20]
            # check if last thing is actually a digit, in case of 5-digit ID
            if not score_name_two[-1].isdigit():
                score_name_two = score_name_two[:-1]
            score_pdf_two = score_name_two + '.pdf'
            score_names.append(score_pdf_two)
    if not score_name[-1].isdigit():
        # make could be a 5-digit ID
        score_name = score_name[:-1]
    score_pdf = score_name + '.pdf' # add to .pdf so we have the actual filename we want to avoid/delete if it's in our thing
    score_names.append(score_pdf) # append this to a list of bad names

# Re-create Dataset

In [2]:
current_dir = '/data1/dbashir/Project/Summer2018/'
topFolder = 'pdf'
resultsFolder = 'results_top50'
intermedFolder = 'results_intermediate'
pdfMiniFolder = 'mini_dataset_pdf'
pngMiniFolder = 'mini_dataset_png'

dataSetSize = 70

resultsPath = os.path.join(topFolder,resultsFolder)
intermedPath = os.path.join(topFolder, intermedFolder)

pdfDir = os.path.join(current_dir, topFolder)
miniPDFDataset = os.path.join(topFolder, pdfMiniFolder)
miniPNGDataset = os.path.join(pdfDir, pngMiniFolder)
pngFromTop = os.path.join(topFolder, pngMiniFolder)
pdfFromTop = os.path.join(pdfDir, pdfMiniFolder)
rotPngFromTop = os.path.join(topFolder, 'mini_dataset_png_rot')

fileList = []

print(pdfDir)

/data1/dbashir/Project/Summer2018/pdf


In [3]:
if not os.path.exists(intermedPath):
    os.makedirs(intermedPath)

In [4]:
if not os.path.exists(miniPDFDataset):
    os.makedirs(miniPDFDataset)

In [5]:
if not os.path.exists(miniPNGDataset):
    os.makedirs(miniPNGDataset)

In [6]:
if not os.path.exists(rotPngFromTop):
    os.makedirs(rotPngFromTop)

In [10]:
#using os.walk, move all the .pdf files to an intermediate folder for EZ access
for subdir, dirs, files in os.walk(resultsPath):
    for pdf_file in files:
        if 'pdf' in pdf_file:
            #move it to intermediate file for shuffling
            print(pdf_file)
            print("subdirectory: " + subdir)
            html_path = os.path.join(subdir, 'html.txt')
            with open(html_path) as f:
                soup = bsoup(f, "html.parser")
                result = soup.find_all('div', class_='we')
                print(result)
            #print("directory: " + str(dirs))
#             pdf_file_path = os.path.join(subdir, pdf_file)
#             final_path = os.path.join(intermedPath, pdf_file)
#             shutil.copy(pdf_file_path, final_path)

42203.pdf
subdirectory: pdf/results_top50/composer/Weber,_Carl_Maria_von/Aufforderung_zum_Tanze,_Op.65_


NameError: name 'bsoup' is not defined